<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [1]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [2]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [3]:
apps = apps.drop(columns=['Type', 'Price', 'Genres', 'Last Updated', 'Current Ver'])

In [4]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','a', regex=True).str.replace('a','.', 1, regex=True).str.replace('a','', regex=True)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})

C:\Users\user\AppData\Local\Temp\ipykernel_10648\2251694235.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\user\AppData\Local\Temp\ipykernel_10648\2251694235.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\user\AppData\Local\Temp\ipykernel_10648\2251694235.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [5]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [6]:
DataProcessing.shuffle(apps)
apps = apps.reset_index(drop=True)

In [7]:
DataProcessing.normalization(apps)

In [8]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.999)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Category,Rating,Reviews,Size,Installs,Content Rating,Android Ver
0,Hitman Sniper,GAME,0.900,9.094579e-03,0.028197,9.999999e-03,Mature 17+,4.10
1,Moto Rider,GAME,0.800,2.565494e-03,0.023162,9.999999e-03,Teen,4.00
2,ChargeHub - Find EV & Tesla Charging Stations,MAPS_AND_NAVIGATION,0.900,1.770842e-05,0.024169,4.999900e-05,Everyone,4.03
3,Frozen Free Fall,FAMILY,0.825,3.506497e-02,0.036254,5.000000e-02,Everyone,4.20
4,Leghe Fantagazzetta,SPORTS,0.775,1.770018e-03,0.026183,9.999990e-04,Everyone,4.40
5,Cardinal Glennon ER Reference,MEDICAL,0.800,4.009455e-07,0.019134,9.990000e-07,Everyone 10+,4.10
6,ESPN,SPORTS,0.800,1.160824e-02,0.036395,9.999999e-03,Everyone 10+,5.00
7,Wi-Fi Auto-connect,COMMUNICATION,0.700,3.349008e-04,0.253776,9.999990e-04,Everyone,1.60


<h2>k-nearest neighbors algorithm</h2>

In [9]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        return math.sqrt(math.pow(v1[2] - v2[2], 2) + math.pow(v1[3] - v2[3], 2) + math.pow(v1[4] - v2[4], 2) + math.pow(v1[5] - v2[5], 2))
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Category'] not in sample['Category'] or dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] *= 2
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        
            
        return dataset.head(10)

<h2>Apps Recommendation</h2>

In [10]:
categoryList = validatingSet['Category'].unique().tolist()
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])

avgApp = pd.DataFrame(columns=['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Content Rating', 'Android Ver'])    
avgApp.loc[0] = ['a', categoryList, avgRating, avgReviews, avgSize, avgInstalls, 'c', maxAndroidVer]
recommendedApps = KNN.clustering(trainingSet, avgApp.loc[0])
recommendedApps

C:\Users\user\AppData\Local\Temp\ipykernel_10648\1050550793.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["distance"] = distances


,App,Category,Rating,Reviews,Size,Installs,Content Rating,Android Ver,distance
4190,iGun Pro -The Original Gun App,GAME,0.800,0.005553,0.054381,0.01,Everyone,2.3,0.012754
7343,Train Racing Games 3D 2 Player,GAME,0.825,0.004285,0.055388,0.01,Everyone,5.0,0.012934
5375,Equestria Girls,FAMILY,0.825,0.008745,0.052367,0.01,Everyone,4.0,0.013049
4215,Equestria Girls,FAMILY,0.825,0.008745,0.052367,0.01,Everyone,4.0,0.013049
1230,Equestria Girls,FAMILY,0.825,0.008750,0.052367,0.01,Everyone,4.0,0.013049
4846,WGT Golf Game by Topgolf,SPORTS,0.825,0.003298,0.056395,0.01,Everyone,4.1,0.013217
2798,Masha and Bear: Cooking Dash,FAMILY,0.825,0.003372,0.053374,0.01,Everyone,4.1,0.013426
1682,COOKING MAMA Let's Cook!,FAMILY,0.825,0.011778,0.058409,0.01,Everyone,4.1,0.013431
3461,COOKING MAMA Let's Cook!,FAMILY,0.825,0.011781,0.058409,0.01,Everyone,4.1,0.013432
6546,Operate Now: Hospital,FAMILY,0.825,0.005677,0.051360,0.01,Teen,4.4,0.013437
